**Run the codes in this sequence: A, B, C, D, E, F, G, H, I, J, K.**

# (A) Import libraries

In [1]:
# os.chdir(r'C:/Users/UWAdmin/Desktop/TEST2/line codes')

from pymeasure.instruments.lighthousephotonics import Sprout #import laser codes
laser_power = Sprout('COM4')
laser_power.adapter.connection.baud_rate = 19200
laser_power.adapter.connection.read_termination = '\r'
laser_power.adapter.connection.write_termination = '\r'
laser_power.power = 0.01


C:\Users\UWAdmin\Anaconda3\lib\site-packages\pyvisa\highlevel.py:3355: FutureWarning: get_instrument is deprecated and will be removed in 1.12, use open_resource instead.
  FutureWarning,


In [2]:
import time
from datetime import timedelta
import rpy2.robjects as robjects

from newportxps import NewportXPS #motion controller
from threading import Thread
# from thread import MotionThread, MirrorThread
import os, sys

import numpy as np
import pandas as pd
from write_utils import write_data_files, get_mean, get_move_y, duplicate_to_dataset

from flipper import mirror #import flipper mirror codes
from spectra import capture_photo #import spectrometer codes

from pathlib import Path


In [3]:
path = Path(r'c:\\Users\\UWAdmin\\Desktop\\_pyControl\\campaigns')
campaign_path = path / "2022-04-26-BATCH-2"
# os.mkdir(campaign_path)
os.chdir(campaign_path)

In [8]:
# write_data_files(path, series=2, nr_random_lines=14, \
#     power_range=(1,1190), time_range=(1050, 7500), pressure_range=(60,350), batch=True)

TODAY's DATE: 2022-04-26
POWER: (1, 1190)
 TIME: (1050, 7500)
 PRESSURE: (60, 350)
Here is the new Campaign Folder:2022-04-26-BATCH-2


# (B) Activate motion controller

In [21]:
XPSD_REMOTE_IP = '192.168.254.254'

xps = NewportXPS(XPSD_REMOTE_IP)
print(xps.status_report())

#pos_all() to get position of all axes
def pos_all():
    for sname, _ in xps.stages.items():
        print('{}: {}'.format(sname, xps.get_stage_position(sname)))
        
xps.kill_group('XYZ')
xps.initialize_allgroups()
xps.home_allgroups()

# XPS host:         192.168.254.254 (XPS-1B81)
# Firmware:         XPS-D-N13019
# Current Time:     Wed Apr 27 08:49:25 2022
# Last Reboot:      Thu Mar  3 06:07:36 2022
# Trajectory Group: XYZ
# Groups and Stages
XYZ (multipleaxesinuse), Status: Not initialized state due to an emergency brake : see positioner error
   XYZ.X (MFA@MFA-CC@XPS-DRV11)
      Hardware Status: ZM low level
      Positioner Errors: OK
   XYZ.Y (MFA@MFA-CC@XPS-DRV11)
      Hardware Status: ZM low level
      Positioner Errors: OK
   XYZ.Z (MFA@MFA-CC@XPS-DRV11)
      Hardware Status: ZM low level
      Positioner Errors: OK


In [1]:
# from pymeasure.instruments.edwards import NXDS
# pump = NXDS('ASRL5::INSTR')
# pump.stop()

C:\Users\UWAdmin\Anaconda3\lib\site-packages\pyvisa\highlevel.py:3355: FutureWarning: get_instrument is deprecated and will be removed in 1.12, use open_resource instead.
  FutureWarning,


In [5]:
# xps.kill_group('XYZ')
# xps.initialize_allgroups()
# xps.home_allgroups()
# xps.move_stage('XYZ.X', 5.5)
# xps.move_stage('XYZ.Y', 6.5)
# xps.move_stage('XYZ.Z', 1.45)
pos_all()

XYZ.X: -1.76994e-05
XYZ.Y: 3.53988e-05
XYZ.Z: 0


In [6]:
mirror("off")

MIRROR DOWN


## (C) Switch on the spectrometer

In [7]:
#Before running this code check that AddInProcess.exe is running (closed if open)
#Close anything related to Light Field

capture_photo("start",2,1,0) # the values 2,1,0 don't matter, they are later used for indexing tasks

## (D) Set chamber pressure
#Used to input new Sample
#Move this before auto focusing

In [8]:
from pressure import close_valve,open_valve,close_all,current_pressure,gopr, to_ambient, to_vacuum, quick_fill
current_pressure()

C:\Users\UWAdmin\Anaconda3\lib\site-packages\pyvisa\highlevel.py:3355: FutureWarning: get_instrument is deprecated and will be removed in 1.12, use open_resource instead.
  FutureWarning,


12

In [105]:
t = 0.5 # in minutes
to_ambient()
time.sleep(1)
to_ambient()
to_vacuum(t)
quick_fill()
time.sleep(1)
to_vacuum(t)


2

# (E) To make sure that the material gets laser beam of max. intensity , focusing of z axis needs to be properly adjusted. With LightField running, use the following code to find z, enter 0 and confirm with yes to end.

In [107]:
mirror('on')

#Iterate for input to find z value
print("Enter z value (enter 0 when done):")
z=''
while z != 0:
    if z == '':
        z = float(input(':'))
    print(z)
    xps.move_stage('XYZ.Z', z)
    xps.move_stage('XYZ.X', 5)
    xps.move_stage('XYZ.Y', 7)
    z = float(input(':'))
    
# mirror('on')
pos_all()
while input('Is LightField Scan Stopped?:(yes/no)')!='yes':
    continue

MIRROR UP
Enter z value (enter 0 when done):
1.8
1.9
2.0
2.1
2.1
2.0
2.01
2.03
1.99
2.0
1.97
1.98
1.99
2.0
2.01
2.0
XYZ.X: 4.999992003
XYZ.Y: 7.0000065036
XYZ.Z: 1.9999968012


# (F) Prepare initial dataset

In [83]:
mirror('off')
# write_data_files(path, nr_random_lines=14,time_range=(1050,7500),pressure_range=(1,30), series=2, batch=True)

MIRROR DOWN


# (G) define Threading class

In [108]:
#Change this to motion_thread
class MotionThread(Thread):
   
    def run(self):
        import pandas as pd
        d = pd.read_csv('foreward.trj',header=None) # scantime 2 (see segment 1 or row1)
        d = d.loc[:, (d != 0).any(axis=0)] # remove last 3 cols with 0
        d.columns = ['ramptime','rampdist','rampvel']
        total_time=np.sum(d['ramptime'])
        
        if d['ramptime'][0]<0.5:
            time.sleep(.5-d['ramptime'][0])
        print("MOTION THREAD\n")
        print("XPS run trajectory")
        xps.run_trajectory('foreward',)
        print('Total time {} in class'.format(total_time))
        time.sleep(total_time)
        print("TOTAL_TIME DONE, LASER 0.05W")
        laser_power.power=0.01
        print("finished and current position is:\n")
        pos_all()

#Change this to mirror_thread
class MirrorThread(Thread):

    def run(self):
        import pandas as pd
        d = pd.read_csv('foreward.trj',header=None) # scantime 2 (see segment 1 or row1)
        d = d.loc[:, (d != 0).any(axis=0)] # remove last 3 cols with 0
        d.columns = ['ramptime','rampdist','rampvel']
        mirror_sleep=d['ramptime'][0]-.5
        print("MIRROR THREAD")
        for i in range(2):
            if i==0:
                print(d['ramptime'][0])
                if d['ramptime'][0]<0.5:
                    print("mirror on")
       
                    start_time = time.monotonic()
                    mirror('on')
       
                    end_time = time.monotonic()
                    print(timedelta(seconds=end_time - start_time))
                    time.sleep(d['ramptime'][1]) #time for linear line    

                else:
                    print('{} > 0.5'.format(d['ramptime'][0]))
                    print('Mirror sleep {} in class'.format(mirror_sleep))
                    time.sleep(mirror_sleep)
                
                    print("mirror on")
                    start_time = time.monotonic()
                    mirror('on')
                    end_time = time.monotonic()
                    print(timedelta(seconds=end_time - start_time))
                    time.sleep(d['ramptime'][1]) #time for linear line
            else:
                mirror('off')
                print("mirror off")

# (H) Define coordinate details

**Takes input for**  
  
-number of lines

-starting position of x

-starting position of y

-vertical gap/step size

In [85]:
lines, start_y, step_y = 7, 6.5, 1
move_y = get_move_y(lines, start_y)
print("Lines will be made at following y axis:", move_y,"\n")

Lines will be made at following y axis: [6.5, 7.5, 8.5, 9.5, 10.5, 11.5, 12.5] 



# (I) After measuring all resistance in dataset. MAKE A COPY OF data.csv. Get resistance mean from the last 2 columns

In [115]:
campaign_path = path / "2022-04-26-BATCH-2"
os.chdir(campaign_path)

# TODO: double-check cols pair are correct and correspond to filled dataset, else overwrite possible
filled_cols = [18,19]
TOTAL_LINES = len(move_y)
SPOTS_MEASURED = 9

for col in filled_cols:
    for move_line in range(TOTAL_LINES):
        save_line = col*TOTAL_LINES + move_line
        steps = save_line * SPOTS_MEASURED
        # get_mean(steps, save_line, spots_measured = SPOTS_MEASURED, target='ratio')
        get_mean(steps, save_line, spots_measured = SPOTS_MEASURED, target='resistance')
        print (f"before: {move_line}, {save_line}, {steps}")

c:\Users\UWAdmin\Desktop\_pyControl\campaigns\2022-04-26-BATCH-2
Filling nans with 0
saving mean 1582.2222222222222 to data.csv
before: 0, 126, 1134
c:\Users\UWAdmin\Desktop\_pyControl\campaigns\2022-04-26-BATCH-2
Filling nans with 0
saving mean 1330.0 to data.csv
before: 1, 127, 1143
c:\Users\UWAdmin\Desktop\_pyControl\campaigns\2022-04-26-BATCH-2
Filling nans with 0
saving mean 898.0 to data.csv
before: 2, 128, 1152
c:\Users\UWAdmin\Desktop\_pyControl\campaigns\2022-04-26-BATCH-2
Filling nans with 0
saving mean 1110.0 to data.csv
before: 3, 129, 1161
c:\Users\UWAdmin\Desktop\_pyControl\campaigns\2022-04-26-BATCH-2
Filling nans with 0
saving mean 1010.0 to data.csv
before: 4, 130, 1170
c:\Users\UWAdmin\Desktop\_pyControl\campaigns\2022-04-26-BATCH-2
Filling nans with 0
saving mean 781.0 to data.csv
before: 5, 131, 1179
c:\Users\UWAdmin\Desktop\_pyControl\campaigns\2022-04-26-BATCH-2
Filling nans with 0
saving mean 1010.0 to data.csv
before: 6, 132, 1188
c:\Users\UWAdmin\Desktop\_pyCon

# (J) Define MBO

In [110]:
robjects.r('''
  experiment<-function(s){

  presToR <- function (x) {(x - 50) / 10}
  presToExp <- function (x) {x * 10 + 50}

  if(s==1){ #BO called first time to initialize and propose first point
  require(XML)

    presToR <- function (x) {(x - 50) / 10}
    presToExp <- function (x) {x * 10 + 50}

    suppressWarnings({suppressMessages({library(mlrMBO)
                                        library(ggplot2)
                                        library(doMC)
                                      })})

    ps = makeParamSet(
      makeIntegerParam("power", lower = 1, upper = 1190),
      makeIntegerParam("time", lower = 1050, upper = 7500),
      makeIntegerParam("pressure", lower = 1, upper = 30, trafo = function(x) x * 10 + 50)
    )

    ctrl = makeMBOControl(y.name = "resistance", propose.points = 14L)
    ctrl = setMBOControlInfill(ctrl, opt = "focussearch", 
                              opt.focussearch.maxit = 10,
                              opt.focussearch.points = 10000,
                              crit = makeMBOInfillCritEI())

    ctrl = setMBOControlMultiPoint(ctrl, method = "cl", cl.lie = min)
    ctrl = setMBOControlTermination(ctrl, iters = 1)

    data = read.csv("data.csv", colClasses = c(NA, NA, NA, "NULL", NA))
    data$pressure <- presToR(data$pressure)

    suppressMessages({opt.state = initSMBO(par.set = ps, 
                                          design = data,
                                          control = ctrl, 
                                          minimize = TRUE, 
                                          noisy = TRUE)})
    print("Proposed parameters:")
    prop = suppressWarnings({proposePoints(opt.state)})
    print(prop$prop.points)
    print(prop$crit.components)
    saveRDS(prop,'./prop.rds')
    saveRDS(opt.state,'./opt.state.rds')
    dummy_ratio = 0
    dummy_resistance = 0
    cat("Expected value (upper bound):\n")
    cat(paste(prop$crit.components$mean, " (",
              prop$crit.components$mean + prop$crit.components$se, ")\n", sep = ""))

    prop$prop.points$pressure <- presToExp(prop$prop.points$pressure)

    x <- data.frame(prop$prop.points, dummy_ratio, dummy_resistance,
                    prop$crit.components$mean,
                    prop$crit.components$mean + prop$crit.components$se, 
                    2*prop$crit.components$mean + prop$crit.components$se)

    x2 <- c(prop$prop.points)

    write.table(x2, file = "data.csv", sep = ",",
                append = TRUE, quote = FALSE,col.names = FALSE, row.names = FALSE)

    write.table(x, file = "plot_data.csv", sep = ",",
                append = TRUE, quote = FALSE,col.names = FALSE, row.names = FALSE)

  }
  else{
    
    presToR <- function (x) {(x-50)/10}
    presToExp <- function (x) {x*10+50}

    prop=readRDS('./prop.rds')
    opt.state=readRDS('./opt.state.rds')
    library(mlrMBO)
    library(R.utils)
    print(prop$crit.components)
    print(prop$prop.points)

    print("Proposed parameters:")
    l2keep <- 14
    nL <- countLines("data.csv")
    s <- read.csv("data.csv", header=FALSE, skip=nL-l2keep)['V5']
    updateSMBO(opt.state, x = prop$prop.points, y = as.list(s$V5))
    prop <- suppressWarnings({proposePoints(opt.state)})
    print(prop$prop.points)
    print(prop$crit.components)
    saveRDS(prop,'./prop.rds')
    saveRDS(opt.state,'./opt.state.rds')

    dummy_ratio = 0
    dummy_resistance = 0
    cat("Expected value (upper bound):\n")
    cat(paste(prop$crit.components$mean, " (",
              prop$crit.components$mean + prop$crit.components$se, ")\n", sep = ""))

    prop$prop.points$pressure <- presToExp(prop$prop.points$pressure)

    x <- data.frame(prop$prop.points, dummy_ratio, dummy_resistance,
                    prop$crit.components$mean,
                    prop$crit.components$mean + prop$crit.components$se, 
                    prop$crit.components$mean + prop$crit.components$mean + prop$crit.components$se)
    x2<-c(prop$prop.points)

    write.table(x2, file = "data.csv", sep = ",",
                append = TRUE, quote = FALSE,col.names = FALSE, row.names = FALSE)

    write.table(x, file = "plot_data.csv", sep = ",",
                append = TRUE, quote = FALSE,col.names = FALSE, row.names = FALSE)

  }}

  ''')

<rpy2.robjects.functions.SignatureTranslatedFunction object at 0x000001E6AFF73A08> [RTYPES.CLOSXP]
R classes: ('function',)

# (K) Measure all resistance. data.csv and dataset.csv should be fully filled and no nan proposed rows are left. If this is not the case, skip to step (M)

# (L) Propose next 14 samples

In [111]:
# campaign_path = path / "2022-04-26-BATCH-2"
# os.chdir(campaign_path)

d = pd.read_csv("data.csv")
rsum = robjects.r['experiment']

NR_LINES_IN_SAMPLE = 126 # HAS TO BE UPDATED +14 EVERY SAMPLE
_NR_PROPOSED_LINES = 14 # don't change this

if len(d['resistance']) == NR_LINES_IN_SAMPLE:
    print(f"First batch proposal with {NR_LINES_IN_SAMPLE} lines to train")
    rsum((1))
    duplicate_to_dataset(nr_proposed_lines=_NR_PROPOSED_LINES, prepattern=False)

elif len(d['resistance']) < NR_LINES_IN_SAMPLE:
    print ("Not enough data points in dataset. Exiting.")
    exit()

else:
    print(f"Further batch proposals of {NR_LINES_IN_SAMPLE} lines")
    proposed = d['resistance'].iloc[-NR_LINES_IN_SAMPLE:]
    rsum((list(proposed)))
    duplicate_to_dataset(nr_proposed_lines=_NR_PROPOSED_LINES)


First batch proposal with 126 lines to train
[1] "Proposed parameters:"
     power time pressure
203    853 5194       30
1932   859 5126       30
2304   451 7500        1
73     852 6448       30
582    376 7500        1
119    851 6252       30
5      413 7500        1
44     403 7500        1
3852   855 5305       30
51     408 7500        1
4      362 7500        1
4025   394 7500        1
187    855 7500       30
37     398 7500        1
         se      mean
1  415.3583 1067.1532
2  354.4227  870.9710
3  521.2237 1043.0716
4  308.1342  699.3578
5  517.3889  871.4160
6  266.2985  676.9892
7  416.6627  769.0682
8  551.2970  895.6334
9  333.6539  702.1116
10 307.6238  671.2931
11 376.3074  721.9683
12 330.9018  687.2480
13 400.6813  731.2709
14 328.7781  693.4894
Expected value (upper bound):
1067.15321166071 (1482.51148384202)
 870.970990452588 (1225.39372320427)
 1043.07159142559 (1564.29529327817)
 699.357813451127 (1007.49204842626)
 871.416016454396 (1388.80491422347)
 676.9891

# (M) Begin the experiment

In [112]:
def main_batch_experiment(startx: float, move_y: list, col: int) -> None:
    """Starts the experiment. Currently works ok with equal length of col1 and col2 lines.
       startx: Starting position on the x-axis
       starty: Starting position on the y-axis
       step_y: The distance between lines on the y-axis
       move_y: List of positions on the y-axis
       col_no: Defines the column number"""

    # line_name_start = col_no*len(move_y) #0,1,   # What the first line # will be named
    # line_name_end = line_name_start + len(move_y) #What the last line # will be named
    
    import pandas as pd
    print(f'LOADING DATA.CSV FOR COLUMN: {col}') 
    df2 = pd.read_csv('data.csv')   # to avoid pd referenced before assignment
    print (df2)
    
    print("BATCH EXPERIMENT IS STARTING\n")
    import time
    time.sleep(5)
    # cc = 0
    # steps = 0
    _MIRROR_MOVE_TIME = .5 # in seconds
    _LINE_LENGTH = 2 # in mm
    VACUUM_TIME = 2 # in minutes

    #set this to zero!!!!!!!
    #starting_line = 4
    STARTING_LINE = 0
    _SPOTS_MEASURED = 9
    TOTAL_LINES = len(move_y) #+STARTING_LINE
    TARGET = 'GD'
    

    print(f"Here's the number of total lines:{TOTAL_LINES}")
    
    for move_line in range(STARTING_LINE,TOTAL_LINES):
        print(f"====== MOVE LINE: {move_line} ======")
        
        save_line = col * TOTAL_LINES + move_line
        steps = save_line * _SPOTS_MEASURED
        print(f"====== SAVE LINE: {save_line} ======")

        # move the axes to their start position
        xps.move_stage('XYZ.Y',move_y[move_line])
        pos_all()
        print("\n\n")
        df2 = pd.read_csv('data.csv')
        plot_data_df = pd.read_csv('plot_data.csv')
        print(f"\n data.csv file before index {save_line}:\n")
        print(df2)
        print(f"\n plot_data.csv file before index {save_line}:\n")
        print(plot_data_df,'\n')
        # line = df2['ratio'].count() #find in data.csv what line number we're on
        
        # set time
        time_of_file = df2['time'][save_line]
        print(f"\nCurrent patterning time is: {time_of_file} ms")
        
        # vacuum-infill
        print("Vacuum-infill in progress...")
        # t = 1 # in minutes
        to_ambient()
        print(f"1st ambient: {current_pressure()}")
        time.sleep(1)
        to_ambient()
        print(f"2nd ambient: {current_pressure()}")
        time.sleep(1)
        to_ambient()
        print(f"3rd ambient: {current_pressure()}")
        time.sleep(5)
        to_vacuum(VACUUM_TIME)
        print(f"1st vac: {current_pressure()}")
        time.sleep(5)
        quick_fill()
        print(f"After quick_fill: {current_pressure()}")
        time.sleep(2)
        to_vacuum(VACUUM_TIME)
        time.sleep(5)
        print(f"2nd vac : {current_pressure()}")

        # set pressure
        pressure_of_file=df2['pressure'][save_line]
        
        for j in range(3):
            gopr(pressure_of_file)
            close_all()
            time.sleep(5)
        print(f"\n Pressure is now :{pressure_of_file}\n")

        xps.define_line_trajectories(start=float(startx),
                                 stop=float(startx)+_LINE_LENGTH,
                                 step=0.01,
                                 scantime=(time_of_file/1000),
                                 axis='X')
        # downloads trajectory defined above
        xps.download_trajectory('foreward.trj')
        import pandas as pd
        d = pd.read_csv('foreward.trj', header=None) # scantime 2 (see segment 1 or row1)
        d = d.loc[:, (d != 0).any(axis=0)] # remove last 3 cols with 0
        d.columns = ['ramptime','rampdist','rampvel']
        print(d)

        total_time = np.sum(d['ramptime'])
        mirror_sleep = d['ramptime'][0] - _MIRROR_MOVE_TIME
        print(f'Total time: {total_time}, Mirror_sleep: {mirror_sleep} in MAIN')

        # power will be set to assigned value, make sure mirror is OFF (!) \
        # and power is given enough time to reach it's value

        power_of_file = df2["power"][save_line]


    ###########################

        # print("#############STARTING PRE-MEASUREMENT ###########\n")
        # check=[float(startx) + .2*_LINE_LENGTH, \
        #         float(startx) + .4*_LINE_LENGTH, \
        #         float(startx) + .6*_LINE_LENGTH]
        # # mm=0
        # for spot in range(3):
        #     print(f"########## spot: {spot} ##############")
        #     xps.move_stage('XYZ.X',check[spot])
        #     print(pos_all())

        #     for iii in range(3):
        #         print(f"############## iteration: {iii} ################")
        #         mirror("on")
        #         #writing G/D of 3 spots in 3 lines inside dataset.csv
        # #         capture_photo("on",spot,line)
        #         GD=capture_photo("first",spot,line,iii)

        #         mirror("off")
        #         time.sleep(5)

    ############################

    # PATTERN EXPERIMENT


        print("Power is now: {} mW".format(power_of_file))
        xps.move_stage('XYZ.X',(float(startx) - d['rampdist'][0]))
        pos_all()
        time.sleep(15)
        print("\n\n")
        mirror("off")
        a=power_of_file
        laser_power.power = (a/0.0994)/1000
        time.sleep(15)
        
        
        print("###### Starting pattern in 5 seconds! #########")
        ##Draw Lines
        time.sleep(10)
        
        import sys
        motion_thread = MotionThread()
        mirror_thread = MirrorThread()

        motion_thread.start()
        mirror_thread.start()

        motion_thread.join()
        mirror_thread.join()

        time.sleep(10)
        print("\n Job done\n")

        ##raman spectra analysis
        laser_power.power=0.01
        time.sleep(15)

    ########################################

    # post patterning raman analysis

        check=[float(startx) + .2*_LINE_LENGTH, \
                float(startx) + .4*_LINE_LENGTH, \
                float(startx) + .6*_LINE_LENGTH]
        # mm=0
        for spot in range(3):
            print(f"############### SPOT {spot} ##################")
            xps.move_stage('XYZ.X',check[spot])
            print(pos_all())

            for iii in range(3):
                print(f"############# iteration {iii} ###############")
                mirror("on")
                #writing G/D of 3 spots in 3 lines inside dataset.csv
        #         capture_photo("on",spot,line)
                print(f"Measuring patterned line for {TARGET}")
                GD = capture_photo("on", spot, save_line, iii, target=TARGET)

                mirror("off")
                time.sleep(5)      

        # if save_line <= TOTAL_LINES:
        print(f"############# LINE {save_line} <= {TOTAL_LINES}, CURRENT_STEP={steps}, TAKING MEAN: #############")
        get_mean(steps, save_line, _SPOTS_MEASURED)

In [113]:
import sys

# path=r'C:\Users\UWAdmin\Desktop\TEST2\Campaigns\2022-04-11-Sample1'
# os.chdir(path)

first_column = int(input("First column number:"))
first_column_verify = int(input("Verify first column number:"))
while first_column !=first_column_verify:
    first_column = int(input("Error, numbers don't match for first column. Re-enter first column number:"))
    first_column_verify = int(input("Verify first column number:"))

second_column = int(input("Second column number:"))
second_column_verify = int(input("Verify second column number:"))

while second_column !=second_column_verify:
    second_column = int(input("Error, numbers don't match for second column. Re-enter second column number:"))
    second_column_verify = int(input("Verify second column number:"))

intensity_check =capture_photo('adjust',0,0,0)
print("---Final Verification of Batch Experiment---")
print("Current Pressure: ", current_pressure())
pos_all()
print("Printer Up To Columns:", filled_cols)
print(f"Print to columns: {first_column} and {second_column}")
print(f"NR_LINES_IN_SAMPLE:{NR_LINES_IN_SAMPLE}")
print(f'Here is the intensity:{intensity_check}')
print("-----------------------------------------")
print("-----------------------------------------")



verify_start = input("Do you approve pressure, intensity, and column numbers (yes/no)")
if verify_start == "yes":
    "Pressure, intensity, and column numbers have been approved. Starting experiment..."
    startx1 = 2.5
    main_batch_experiment(startx1, move_y, col=first_column)

    startx2 = 5.5
    main_batch_experiment(startx2, move_y, col=second_column)
else:
    sys.exit()

MIRROR UP
 
MIRROR DOWN
---Final Verification of Batch Experiment---
Current Pressure:  14
XYZ.X: 4.999992003
XYZ.Y: 7.0000065036
XYZ.Z: 1.9999968012
Printer Up To Columns: [16, 17]
Print to columns: 18 and 19
NR_LINES_IN_SAMPLE:126
Here is the intensity:203.0
-----------------------------------------
-----------------------------------------
LOADING DATA.CSV FOR COLUMN: 18
     power  time  pressure     ratio   resistance
0     1047  4702       280  1.797723  1851.111111
1      264  3365       320  2.485434  1480.000000
2      190  2965       210  1.116985  1565.555556
3      198  2144       140  1.002617  1440.000000
4     1124  3125       210  2.826222  1140.000000
5      590  6611       280  1.244976   766.000000
6      455  1760       260  0.821883   987.000000
7      212  3544       170  1.943867  1941.111111
8      731  4421       270  1.389385  1730.000000
9     1010  5252        70  1.494728  1785.555556
10     660  1384       160  3.955769  1692.222222
11     301  7099       

In [114]:
to_ambient()

21

In [ ]:
# # campaign_path = path / "2022-04-18-BATCH-1"
# # os.chdir(campaign_path)

# col = 2
# TOTAL_LINES = len(move_y)
# for move_line in range(TOTAL_LINES):
#         # print("For this print this is line #: ",move_line)
#         # print("Experiment Line number: " ,move_line)
        
#         save_line = col*TOTAL_LINES + move_line
#         steps = save_line * 9
#         print (f"before: {move_line}, {save_line}, {steps}")
        
#         # print (f"after : {move_line}, {save_line}, {steps}")

before: 0, 14, 126
before: 1, 15, 135
before: 2, 16, 144
before: 3, 17, 153
before: 4, 18, 162
before: 5, 19, 171
before: 6, 20, 180


In [ ]:
# campaign_path = path / "2022-04-18-BATCH-1"
# os.chdir(campaign_path)

# SPOTS_MEASURED = 9
# save_line, steps = 14, 126
# get_mean(steps, save_line, spots_measured = SPOTS_MEASURED)

c:\Users\UWAdmin\Desktop\_pyControl\campaigns\2022-04-18-BATCH-1


0.8577710131111111